Webscraper for autofilling website link for pho stores by searching store infomation in Google

In [40]:
import bs4
import csv
import time
import requests
import pandas as pd

import the source file and convert to Pandas

In [41]:
df = pd.read_excel('pho_fever_fixed.xlsx')
df

,Name,Address,Phone number,Other (website)
0,Pho Mailee Vietnamese Noodle Soup,"3020 Minnesota Dr,Anchorage, AK 99503",(907) 258-4746,NaN
1,Saigon Pho,"1381 N Alma School Rd,Chandler, AZ 85224",(480) 786-8828,NaN
2,Pho Bang Restaurant,"1702 W Camelback Rd,Phoenix, AZ 85015",(602) 433-9440,NaN
3,Scottsdale Pho,"9201 E Mountain View Rd ,Scottsdale, AZ 85258",(480) 551-7900,NaN
4,Pho Nhat,"933 E University Dr,Tempe, AZ 85281",(480) 967-1888,NaN
...,...,...,...,...
3262,Pho 20,"47 e Beaver Creek Blvd,Avon, CO 81620",() -,NaN
3263,Pho Ly,"2141 US Hwy 2 E,Kalispell, MT 59901",() -,NaN
3264,Pho Saigon Noodle House 2,"4907 S. State St.,Murray, UT 84107",(801) 590-8277,NaN
3265,Saigon Dragon,"136th & Pecos,Broomfield, CO",(303) 252-1688,NaN


In [42]:
df.iloc[0]['Address']

'3020 Minnesota Dr,Anchorage, AK 99503'

In [43]:
def findWebSite(query):
    
    header={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
    # query="Pho 79	29 S Garfield Ave,Alhambra, CA 91801"
    res = requests.get(f"https://www.google.com/search?q={query}",allow_redirects=False,headers=header)
    soup = bs4.BeautifulSoup(res.text,"html.parser")
    # print(soup.prettify())

    buttons_res=soup.findAll('a',{"class":"ab_button"},text='Website')
    title_res = soup.findAll('h2',{"data-attrid":"title"})
    biz_info=soup.find('div',{"data-attrid":"kc:/local:one line summary"})
    if title_res:
        for t in title_res:
            title = t.getText()
            print(title)
    else:
        title = 'Not a valid business name'
        print(title)
    if (buttons_res):# find website link

        for button in buttons_res:
            website_link=(button['href'])
            print(website_link)
    else:
        print('No website link found')
        website_link = 'N/A'
    if (biz_info):
        biz_type = biz_info.findChildren("span")[-1] or "Not a Business"
        print(biz_type)
    else:
        print("N/A")
        biz_type="Not a business"
    time.sleep(2.5)
    return (title,website_link,biz_type) 


In [44]:
df['Business Type']='Unknown'

In [45]:
df

,Name,Address,Phone number,Other (website),Business Type
0,Pho Mailee Vietnamese Noodle Soup,"3020 Minnesota Dr,Anchorage, AK 99503",(907) 258-4746,NaN,Unknown
1,Saigon Pho,"1381 N Alma School Rd,Chandler, AZ 85224",(480) 786-8828,NaN,Unknown
2,Pho Bang Restaurant,"1702 W Camelback Rd,Phoenix, AZ 85015",(602) 433-9440,NaN,Unknown
3,Scottsdale Pho,"9201 E Mountain View Rd ,Scottsdale, AZ 85258",(480) 551-7900,NaN,Unknown
4,Pho Nhat,"933 E University Dr,Tempe, AZ 85281",(480) 967-1888,NaN,Unknown
...,...,...,...,...,...
3262,Pho 20,"47 e Beaver Creek Blvd,Avon, CO 81620",() -,NaN,Unknown
3263,Pho Ly,"2141 US Hwy 2 E,Kalispell, MT 59901",() -,NaN,Unknown
3264,Pho Saigon Noodle House 2,"4907 S. State St.,Murray, UT 84107",(801) 590-8277,NaN,Unknown
3265,Saigon Dragon,"136th & Pecos,Broomfield, CO",(303) 252-1688,NaN,Unknown


In [46]:
len(df)

3267

In [47]:
df2 = pd.DataFrame(columns=["Name","Address",'Webpage','Phone number','Business type'])


In [48]:
for i in range (0,len(df)):
# for i in range (0,5):    
    query = (df.iloc[i]['Name'])+' '+(df.iloc[i]['Address'])
    print(query)
    res=findWebSite(query)
    df.iat[i,0]=[res[0]]
    df.iat[i,3]=[res[1]]
    df.iat[i,4]=[res[2]]
  

Pho Mailee Vietnamese Noodle Soup 3020 Minnesota Dr,Anchorage, AK 99503
Pho Mailee Vietnamese Noodle
No website link found
N/A
Saigon Pho 1381 N Alma School Rd,Chandler, AZ 85224
Saigon Pho & Seafood Restaurant
http://www.saigonphoaz.com/
<span class="YhemCb">Vietnamese restaurant</span>
Pho Bang Restaurant 1702 W Camelback Rd,Phoenix, AZ 85015
Phở Thành Restaurant
No website link found
<span class="YhemCb">Vietnamese restaurant</span>
Scottsdale Pho 9201 E Mountain View Rd ,Scottsdale, AZ 85258
SimonMed Imaging - Scottsdale Mountain View
https://www.simonmed.com/locations/az/scottsdale-mountain-view/
<span class="YhemCb">Medical diagnostic imaging center in Scottsdale, Arizona</span>
Pho Nhat 933 E University Dr,Tempe, AZ 85281
Phở Nhật Restaurant
No website link found
<span class="YhemCb">Vietnamese restaurant</span>
Pho Quang 37 W Baseline Rd ,Tempe, AZ 85283
BoSa Donuts
No website link found
<span class="YhemCb">Donut shop</span>
Pho Thu 2226 Stone Ave,Tucson, AZ 85701
Pho #1
No we

Function to search Google 

In [49]:
df.head(5)

,Name,Address,Phone number,Other (website),Business Type
0,[Pho Mailee Vietnamese Noodle],"3020 Minnesota Dr,Anchorage, AK 99503",(907) 258-4746,[N/A],[Not a business]
1,[Saigon Pho & Seafood Restaurant],"1381 N Alma School Rd,Chandler, AZ 85224",(480) 786-8828,[http://www.saigonphoaz.com/],[[Vietnamese restaurant]]
2,[Phở Thành Restaurant],"1702 W Camelback Rd,Phoenix, AZ 85015",(602) 433-9440,[N/A],[[Vietnamese restaurant]]
3,[SimonMed Imaging - Scottsdale Mountain View],"9201 E Mountain View Rd ,Scottsdale, AZ 85258",(480) 551-7900,[https://www.simonmed.com/locations/az/scottsd...,[[Medical diagnostic imaging center in Scottsd...
4,[Phở Nhật Restaurant],"933 E University Dr,Tempe, AZ 85281",(480) 967-1888,[N/A],[[Vietnamese restaurant]]


In [50]:
df.to_excel('pho_fever_fixed_2.xlsx')

In [51]:
# header={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
# query="Pho 79	29 S Garfield Ave,Alhambra, CA 91801"
# res = requests.get(f"https://www.google.com/search?q={query}",allow_redirects=False,headers=header)
# soup = bs4.BeautifulSoup(res.text,"html.parser")
# # print(soup.prettify())

# buttons_res=soup.findAll('a',{"class":"ab_button"},text='Website')
# title_res = soup.findAll('h2',{"data-attrid":"title"})
# biz_info=soup.find('div',{"data-attrid":"kc:/local:one line summary"})
    

# for t in title_res:
#     title = t.getText()
#     print(title)
# if (buttons_res):# find website link

#     for button in buttons_res:
#         website_link=(button['href'])
#         print(website_link)
# else:
#     print('No website link found')
# if (biz_info):
#     biz_type = biz_info.findChildren("span")[2]
#     for item in biz_type:
#         print(item)
# else:
#     print("N/A")


In [99]:
df2 = pd.read_excel('pho-places-datamined-phofever.xlsx')

In [100]:
df2

,Name,Address,Phone number,Other (website),Business Type
0,Saigon Pho & Seafood Restaurant,"1381 N Alma School Rd,Chandler, AZ 85224",(480) 786-8828,http://www.saigonphoaz.com/,Vietnamese restaurant
1,Phở Thành Restaurant,"1702 W Camelback Rd,Phoenix, AZ 85015",(602) 433-9440,NaN,Vietnamese restaurant
2,Phở Nhật Restaurant,"933 E University Dr,Tempe, AZ 85281",(480) 967-1888,NaN,Vietnamese restaurant
3,BoSa Donuts,"37 W Baseline Rd ,Tempe, AZ 85283",(480) 491-8700,NaN,Donut shop
4,Pho #1,"2226 Stone Ave,Tucson, AZ 85701",(520) 670-1705,NaN,Vietnamese restaurant
...,...,...,...,...,...
2316,Phở 20,"47 e Beaver Creek Blvd,Avon, CO 81620",() -,http://www.pho20avon.com/,Vietnamese restaurant
2317,Pho Ly Asian Cuisine,"2141 US Hwy 2 E,Kalispell, MT 59901",() -,NaN,Vietnamese restaurant
2318,Phở Saigon Noodle House,"4907 S. State St.,Murray, UT 84107",(801) 590-8277,NaN,Vietnamese restaurant
2319,Saigon Dragon Asian Cuisine,"136th & Pecos,Broomfield, CO",(303) 252-1688,http://saigondragonco.com/,Asian restaurant


14

In [101]:

header={ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
# query="Pho 79	29 S Garfield Ave,Alhambra, CA 91801"
for i in range (0,len(df2)):

    query = (df2.iloc[i]['Name']+' '+df2.iloc[i]['Address'])
    print(query)
    res = requests.get(f"https://www.google.com/search?q={query}",allow_redirects=False,headers=header)
    soup = bs4.BeautifulSoup(res.text,"html.parser")
    # print(soup.prettify())


    phone_number = soup.find('div',{"data-attrid":"kc:/collection/knowledge_panels/has_phone:phone"})
    is_permanently_closed=soup.find('div',{"data-attrid":"kc:/local:permanently closed"})

    if (is_permanently_closed):
        df2.iat[i,4]='Permanently Closed'
        print('Permanently closed')
    else:
        if ((len(str(df2.iat[i,2]))<14) and (phone_number)):
            df2.iat[i,2]=phone_number.get_text()
            print(phone_number)
        else: 
            print('Phone number not replaced')
    time.sleep(2)
    

Saigon Pho & Seafood Restaurant 1381 N Alma School Rd,Chandler, AZ 85224
Phone number not replaced
Phở Thành Restaurant 1702 W Camelback Rd,Phoenix, AZ 85015
Phone number not replaced
Phở Nhật Restaurant 933 E University Dr,Tempe, AZ 85281
Phone number not replaced
BoSa Donuts 37 W Baseline Rd ,Tempe, AZ 85283
Phone number not replaced
Pho #1 2226 Stone Ave,Tucson, AZ 85701
Phone number not replaced
Phở Cần Thơ Vietnamese Cuisine 2744 N Campbell Ave,Tucson, AZ 85719
Phone number not replaced
Saigon Eden 29 S Garfield Ave,Alhambra, CA 91801
Phone number not replaced
Phở 94 Restaurant 619 N Euclid St,Anaheim, CA 92801
Phone number not replaced
Pho Pioneer Restaurant 17701 Pioneer Blvd,Artesia, CA 90701
Phone number not replaced
Pho Tasty 19117 Pioneer Blvd,Artesia, CA 90701
Phone number not replaced
Pho Huong Que 2126 Center St,Berkeley, CA 94704
Phone number not replaced
Flavor Pho 11444 South St,Cerritos, CA 90703
Phone number not replaced
Pho Grand Vietnamese Noodle & Grill 5368 River

In [103]:
df2.to_excel('pho_fever_fixed_4.xlsx')